## Import Libraries and Initialize Binance API:

In [9]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import datetime
import os
import sys
import backtrader as bt
import ccxt
import pandas as pd
import matplotlib.pyplot as plt

# Initialize Binance API
binance = ccxt.binance()

##  Fetching Data from Binance

In [15]:
def fetch_ohlcv(symbol, timeframe, since=None, limit=1000):
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
    return ohlcv

def ohlcv_to_dataframe(ohlcv):
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

def fetch_data_for_period(symbol, timeframe, start_date, end_date, limit=1000):
    all_data = []
    since = binance.parse8601(start_date)
    end_time = binance.parse8601(end_date)
    while since < end_time:
        ohlcv = fetch_ohlcv(symbol, timeframe, since, limit)
        if not ohlcv:
            break
        all_data.extend(ohlcv)
        since = ohlcv[-1][0]  # Set since to the last timestamp fetched
        since += 1  # Move to the next millisecond to avoid duplication
    return all_data


## Converting Data to DataFrame and Saving as CSV

In [17]:
df = ohlcv_to_dataframe(ohlcv)
csv_path = '../data/binance_btc_usdt_6months.csv'
df.to_csv(csv_path, index=False)


## Strategy Definition

In [18]:
class TestStrategy(bt.Strategy):
    params = (('maperiod', 15),)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)
        # Additional indicators for plotting
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25, subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price, order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price, order.executed.value, order.executed.comm))
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        if self.order:
            return
        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.dataclose[0] < self.sma[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()


## Running the Strategy

In [24]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy)
    data = bt.feeds.GenericCSVData(
        dataname='binance_btc_usdt_6months.csv',
        dtformat=('%Y-%m-%d %H:%M:%S'),
        datetime=0,
        open=1,
        high=2,
        low=3,
        close=4,
        volume=5,
        openinterest=-1,
        timeframe=bt.TimeFrame.Minutes,
        compression=60
    )
    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=5)
    cerebro.broker.setcommission(commission=0.0)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # Plot settings
    plt.rcParams['figure.figsize'] = [15, 12]
    plt.rcParams.update({'font.size': 12})
    cerebro.plot(iplot=False)


Starting Portfolio Value: 100000.00
2023-01-02, Close, 16717.50
2023-01-02, BUY CREATE, 16717.50
2023-01-02, BUY EXECUTED, Price: 16717.50, Cost: 83587.50, Comm 0.00
2023-01-02, Close, 16727.95
2023-01-02, Close, 16735.11
2023-01-02, Close, 16721.54
2023-01-02, Close, 16694.78
2023-01-02, Close, 16698.69
2023-01-02, Close, 16734.66
2023-01-02, Close, 16710.88
2023-01-02, Close, 16715.42
2023-01-02, Close, 16723.81
2023-01-02, Close, 16737.56
2023-01-02, Close, 16728.52
2023-01-02, Close, 16757.76
2023-01-02, Close, 16698.40
2023-01-02, SELL CREATE, 16698.40
2023-01-02, SELL EXECUTED, Price: 16698.40, Cost: 83587.50, Comm 0.00
2023-01-02, OPERATION PROFIT, GROSS -95.50, NET -95.50
2023-01-02, Close, 16672.87
2023-01-03, Close, 16695.59
2023-01-03, Close, 16677.60
2023-01-03, Close, 16674.03
2023-01-03, Close, 16691.81
2023-01-03, Close, 16685.64
2023-01-03, Close, 16725.50
2023-01-03, BUY CREATE, 16725.50
2023-01-03, BUY EXECUTED, Price: 16725.50, Cost: 83627.50, Comm 0.00
2023-01-03, C

<IPython.core.display.Javascript object>

## Strategy and Indicators
### Strategy:
- The strategy implemented is based on a Simple Moving Average (SMA) crossover:

    - Buy Condition: The strategy attempts to buy when the closing price of the asset is above the SMA (15-period).
    - Sell Condition: The strategy attempts to sell when the closing price of the asset is below the SMA (15-period).

### Indicators:    

- Simple Moving Average (SMA): A 15-period SMA is used to determine the buy/sell signals.
- Additional indicators are included for plotting and analysis but are not used in the buy/sell decision logic:
    - Exponential Moving Average (EMA): 25-period.
    - Weighted Moving Average (WMA): 25-period.
    - Stochastic Slow: Stochastic indicator to measure momentum.
    - MACD Histogram: Moving Average Convergence Divergence for trend-following.
    - Relative Strength Index (RSI): To measure the speed and change of price movements.
    - ATR (Average True Range): Measures volatility.